In [4]:
!pip install textblob
!pip install textsearch
!pip install contractions
!pip install afinn
import nltk
nltk.download('all')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53430 sha256=5d9c66f839a9ecd1ebd16a9fc6d170a292f02c914f9c71e3e95fd7801789491f
  Stored in directory: /root/.cache/pip/wheels/b0/05/90/43f79196199a138fb486902fceca30a2d1b5228e6d2db8eb90
Successfully built afinn


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [55]:
#Beautiful soup
from bs4 import BeautifulSoup
import tqdm
import re
import unicodedata
import numpy as np
import contractions
import pandas as pd

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
import unicodedata
def strip_html_tags(text):
  soup = BeautifulSoup(text,'html.parser')
  stripped_text = soup.get_text()
  stripped_text = re.sub("[\r|\n|\r\n]+",'\n',stripped_text)
  return stripped_text

In [8]:
stop_words = nltk.corpus.stopwords.words('english')

In [9]:
def remove_accented_characters(text):
  text = unicodedata.normalize("NFKD",text).encode('ascii','ignore').decode('utf-8','ignore')
  return text

In [49]:
import string
import re
import tqdm
def pre_processing(doc):
  norm_corpus = []
  for text in tqdm.tqdm(doc):
    text = text.lower()
    text = contractions.fix(text)
    text = re.sub(r'[^a-zA-Z0-9\s]',' ',text,re.I)
    tokens = nltk.word_tokenize(text)
    text = text.strip
    filtered_tokens = [token for token in tokens if token not in stop_words]
    text = " ".join(filtered_tokens)
    text = re.sub(' +',' ',text)
    norm_corpus.append(text)
  return norm_corpus


In [50]:
corpus= ["Inceptez is a great place to learn data science and big data",
         "data is called the next oil, so learn data science",
         "big data is the science of dealing with data at a scale"
         "data science can be simple filtering or sorting.",
         "data science is easy to learn"]

In [51]:
corpus = np.array(corpus)
print(corpus)
normalized_corpus = pre_processing(corpus)

['Inceptez is a great place to learn data science and big data'
 'data is called the next oil, so learn data science'
 'big data is the science of dealing with data at a scaledata science can be simple filtering or sorting.'
 'data science is easy to learn']


100%|██████████| 4/4 [00:00<00:00, 1916.96it/s]


In [52]:
normalized_corpus

['inceptez great place learn data science big data',
 'data called next oil learn data science',
 'big data science dealing data scaledata science simple filtering sorting',
 'data science easy learn']

In [56]:
cv = CountVectorizer()
cv_matrix = cv.fit_transform(normalized_corpus)
cv_matrix = cv_matrix.toarray()
vocab = cv.get_feature_names_out()
pd.DataFrame(cv_matrix, columns = vocab)

,big,called,data,dealing,easy,filtering,great,inceptez,learn,next,oil,place,scaledata,science,simple,sorting
0,1,0,2,0,0,0,1,1,1,0,0,1,0,1,0,0
1,0,1,2,0,0,0,0,0,1,1,1,0,0,1,0,0
2,1,0,2,1,0,1,0,0,0,0,0,0,1,2,1,1
3,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0


In [57]:
cv = CountVectorizer(binary = True)
cv_matrix = cv.fit_transform(normalized_corpus)
cv_matrix = cv_matrix.toarray()
vocab = cv.get_feature_names_out()
pd.DataFrame(cv_matrix, columns = vocab)

,big,called,data,dealing,easy,filtering,great,inceptez,learn,next,oil,place,scaledata,science,simple,sorting
0,1,0,1,0,0,0,1,1,1,0,0,1,0,1,0,0
1,0,1,1,0,0,0,0,0,1,1,1,0,0,1,0,0
2,1,0,1,1,0,1,0,0,0,0,0,0,1,1,1,1
3,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0


In [59]:
#ngram vectors
cv = CountVectorizer(ngram_range = (2,2))
cv_matrix = cv.fit_transform(normalized_corpus)
cv_matrix = cv_matrix.toarray()
vocab = cv.get_feature_names_out()
pd.DataFrame(cv_matrix, columns = vocab)

,big data,called next,data called,data scaledata,data science,dealing data,easy learn,filtering sorting,great place,inceptez great,learn data,next oil,oil learn,place learn,scaledata science,science big,science dealing,science easy,science simple,simple filtering
0,1,0,0,0,1,0,0,0,1,1,1,0,0,1,0,1,0,0,0,0
1,0,1,1,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0
2,1,0,0,1,1,1,0,1,0,0,0,0,0,0,1,0,1,0,1,1
3,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0


In [60]:
#ngram vectors
cv = CountVectorizer(ngram_range = (2,3))
cv_matrix = cv.fit_transform(normalized_corpus)
cv_matrix = cv_matrix.toarray()
vocab = cv.get_feature_names_out()
pd.DataFrame(cv_matrix, columns = vocab)

,big data,big data science,called next,called next oil,data called,data called next,data scaledata,data scaledata science,data science,data science big,...,science big,science big data,science dealing,science dealing data,science easy,science easy learn,science simple,science simple filtering,simple filtering,simple filtering sorting
0,1,0,0,0,0,0,0,0,1,1,...,1,1,0,0,0,0,0,0,0,0
1,0,0,1,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,1,1,1,0,...,0,0,1,1,0,0,1,1,1,1
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,0


In [61]:
#ngram vectors
cv = CountVectorizer(ngram_range = (1,3))
cv_matrix = cv.fit_transform(normalized_corpus)
cv_matrix = cv_matrix.toarray()
vocab = cv.get_feature_names_out()
pd.DataFrame(cv_matrix, columns = vocab)

,big,big data,big data science,called,called next,called next oil,data,data called,data called next,data scaledata,...,science dealing,science dealing data,science easy,science easy learn,science simple,science simple filtering,simple,simple filtering,simple filtering sorting,sorting
0,1,1,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,1,2,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,2,0,0,1,...,1,1,0,0,1,1,1,1,1,1
3,0,0,0,0,0,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,0


In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer()
tv_matrix = tv.fit_transform(normalized_corpus)
tv_matrix = tv_matrix.toarray()
vocab = tv.get_feature_names_out()
pd.DataFrame(tv_matrix, columns = vocab)

,big,called,data,dealing,easy,filtering,great,inceptez,learn,next,oil,place,scaledata,science,simple,sorting
0,0.339574,0.000000,0.449521,0.000000,0.000000,0.000000,0.430707,0.430707,0.274915,0.000000,0.000000,0.430707,0.000000,0.224761,0.000000,0.000000
1,0.000000,0.457916,0.477920,0.000000,0.000000,0.000000,0.000000,0.000000,0.292282,0.457916,0.457916,0.000000,0.000000,0.238960,0.000000,0.000000
2,0.282294,0.000000,0.373695,0.358054,0.000000,0.358054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.358054,0.373695,0.358054,0.358054
3,0.000000,0.000000,0.373502,0.000000,0.715739,0.000000,0.000000,0.000000,0.456847,0.000000,0.000000,0.000000,0.000000,0.373502,0.000000,0.000000


In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(ngram_range = (1,2))
tv_matrix = tv.fit_transform(normalized_corpus)
tv_matrix = tv_matrix.toarray()
vocab = tv.get_feature_names_out()
pd.DataFrame(tv_matrix, columns = vocab)

,big,big data,called,called next,data,data called,data scaledata,data science,dealing,dealing data,...,scaledata,scaledata science,science,science big,science dealing,science easy,science simple,simple,simple filtering,sorting
0,0.238736,0.238736,0.000000,0.000000,0.316034,0.000000,0.000000,0.158017,0.000000,0.000000,...,0.000000,0.000000,0.158017,0.302807,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.321696,0.321696,0.335749,0.321696,0.000000,0.167874,0.000000,0.000000,...,0.000000,0.000000,0.167874,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.199015,0.199015,0.000000,0.000000,0.263452,0.000000,0.252425,0.131726,0.252425,0.252425,...,0.252425,0.252425,0.263452,0.000000,0.252425,0.000000,0.252425,0.252425,0.252425,0.252425
3,0.000000,0.000000,0.000000,0.000000,0.253897,0.000000,0.000000,0.253897,0.000000,0.000000,...,0.000000,0.000000,0.253897,0.000000,0.000000,0.486541,0.000000,0.000000,0.000000,0.000000


In [65]:
movie_reviews = pd.read_csv("movie_reviews.csv")

In [66]:
movie_reviews.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [67]:
def pre_processing_movies(doc):
  norm_corpus = []
  for text in tqdm.tqdm(doc):
    text = text.lower()
    text = strip_html_tags(text)
    text = remove_accented_characters(text)
    text = contractions.fix(text)
    text = re.sub(r'[^a-zA-Z0-9\s]',' ',text,re.I)
    text = text.translate(str.maketrans(' ',' ',string.punctuation))
    tokens = nltk.word_tokenize(text)
    text = text.strip
    filtered_tokens = [token for token in tokens if token not in stop_words]
    text = " ".join(filtered_tokens)
    text = re.sub(' +',' ',text)
    norm_corpus.append(text)
  return norm_corpus


In [69]:
movie_reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [70]:
train_reviews= movie_reviews['review'][:35000]
train_senti = movie_reviews['sentiment'][:35000]
test_reviews= movie_reviews['review'][35000:]
test_senti = movie_reviews['sentiment'][35000:]


In [71]:
normalized_train = pre_processing_movies(train_reviews)
normalized_test = pre_processing_movies(test_reviews)

  2%|▏         | 852/35000 [00:01<00:41, 821.74it/s]<ipython-input-7-5c8602756ae5>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text,'html.parser')
100%|██████████| 15000/15000 [00:29<00:00, 509.42it/s]


In [89]:
train_senti = np.array(train_senti)

In [72]:
normalized_train = np.array(normalized_train)
normalized_test = np.array(normalized_test)

In [96]:
len(normalized_train)

35000

In [97]:
from sklearn.feature_extraction.text import CountVectorizer


In [112]:
cv = CountVectorizer(min_df = 5, max_df = 1.0,binary = False, ngram_range = (1,2))
normalized_train_cv = cv.fit_transform(normalized_train)
normalized_test_cv = cv.transform(normalized_test)

In [102]:
normalized_train_cv.shape

(35000, 2607643)

In [103]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty = 'l2',max_iter = 600,random_state = 24)
lr.fit(normalized_train_cv,train_senti)

LogisticRegression(max_iter=600, random_state=24)

In [104]:
lr_bow = lr.predict(normalized_test_cv)

In [106]:
from sklearn.metrics import confusion_matrix, classification_report

In [111]:
labels = ['negative','positive']

print(classification_report(test_senti, lr_bow))
pd.DataFrame(confusion_matrix(test_senti, lr_bow),index = labels,columns = labels)

              precision    recall  f1-score   support

    negative       0.90      0.89      0.90      7490
    positive       0.90      0.90      0.90      7510

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



,negative,positive
negative,6695,795
positive,719,6791


In [92]:
normalized_train_cv.shape

(4, 36)

In [ ]:
1. Information to the vector
2. Losses